- the purpose of this Py script is to count separately the number of human vs Neanderthal loci found in the sequencing data. 
- also to conclude if the sampled individual is related closer to modern humans or to Neanderthals.

*Graphical abstract for the code* |  *The pseudocode for the script*|
:-----:     |:--------------:|
<img src="graphical_abstr_count1.jpg" alt="graphical abstract" title="Graphical abstract for the code" width="400" /> | <img src="script_pseudocode.jpg" alt="pseudocode" title="Graphical abstract for the code" width="700" /> |

<img src="script_pseudocode1.jpg" width="400" />   
*More pseudocode example* 

## Gather the necessary files
* on Uppmax, the original files are in:  
/proj/snic2019-8-150/nobackup/private/Mirela/neanderthal_project/data/raw_internal  
/home/miba8458/LnktoWorkingFolder/UppmaxDataLnk/data/raw_external/  

* to work on them, I have copied the files to this location  
/home/miba8458/LnktoWorkingFolder    
/home/miba8458/private/workingFolder  

* the orinal files to be used as control (known if the aDNA they contain was either human or Neanderthal)  
/proj/snic2019-8-150/nobackup/private/Mirela/neanderthal_project/data/raw_external  

[ ] the .bam sequencing files, filtered and sorted  
[ ] the .ibam files, containing the indexes of the .bam files  
[ ] the list with the genomic locations known to differ between modern humans and Neanderthals   
  (#Ref paper: Phylogenetic analysis in 'Nuclear DNA sequences from the Middle Pleistocene Sima de los Huesos hominins' - Meyer, Nature, 2016    
   #DL link: https://bioinf.eva.mpg.de/sima/)

## explore the files 
- to see what is needs to be extracted  
### A) the reference list:   
    - col 1 - chr nr (chromosome on hg19/GRCh37) 
    - col 2 - genomic location (position on hg19/GRCh37) - unique position?
    - col 3 - ancestral state present in the chimpanzee, bonobo, gorilla, orangutan and rhesus macaque reference sequences  
    - col 4 - derived state  
    - col 5 - of which species is the substitution representative of (indicates the presence of the derived state Altai Neandertal, Denisovan or Mbuti. "all" means that all three were derived)   
        - Neandertal  
        - Denisova  
        - Human  (<- Mbuti genome)
        - Human-Denisova   
        - Neandertal-Human   
        - Neandertal-Denisova  
        - all  
  
(to obtain the values of col 5: `less -c sima_informative_sites.tab | awk '{a[$5]++} END {for (n in a) print n, a[n]}'`)

#bash code  
#do this in Py when having time   
`less sima_informative_sites.tab | head -5` - to read the file and display the first 5 rows  
10	94488	G	A	all	 
10	95365	T	C	all	 
10	95830	T	C	all	   
10	97555	G	A	all	 
10	97647	G	T	all	 

`sed -n '55,60p' <sima_informative_sites.tab` - to display the rows 55 to 60 of the file   
10	147482	T	C	Neandertal 	
10	147544	G	A	all	 
10	147812	A	G	all	 
10	148000	A	G	all	 
10	148199	C	T	all	 
10	148542	G	A	Denisova		 

### B) the file containing the genome reference 
* we are using the hs37d5.fa reference genome
* When I printed out the first few lines, I got lots of 'NNNNNNNNNNNNN'. Actually lines 2-168 are only like that.
* looked further down, and I printed lines 1 and 170:  
`zless hs37d5.fa.gz|sed -n '1p;170p`
>1 dna:chromosome chromosome:GRCh37:1:1:249250621:1
ACCCTAACCCTAACCCTAACCCTAACCCAACCCTAACCCTAACCCTAACCCTAACCCTAA

### C) the files containing the sequences
- .bam files with 11 mandatory columns and 1 optional
- in the .bam file, the columns that I am interested in are:
    - col 3: reference sequence  
    - col 4: position number (the starting position of the read)  
    - col 6: alphanumeric; if only 'M', 'I', 'S', '=' and/ or 'X', then the length of the sequence is the sum of the numbers in this string   
  
(to look at the first 5 rows of the *alignment* and ignore the header of the .bam:    
`samtools view jar001-b1e1l1p1_CATGCTC_L004_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam | head -5`)  
    
(to print the min and max values of col 4 - in the 2 .bam files I have:   
`samtools view jar001-b1e1l1p1_CATGCTC_L004_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam | awk 'BEGIN{a=1000}{if ($4<0+a) a=$4} END{print a}'`  
result - 5  | 178  
`samtools view jar001-b1e1l1p1_CATGCTC_L004_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam | awk 'BEGIN{a=   0}{if ($4>0+a) a=$4} END{print a}'`  
result - 249 240 549 | 249 239 780)  
   
(to look at a few values around the max value of col 4, I change the value and use it as a int instead of a string:)  
`samtools view jar002-b1e1l1p1_ACGCAAC_L005_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam | awk '{if ($4 > 249200000) print $0;}'`  
  
(to look at the total number of reads:)
`samtools idxstats jar001-b1e1l1p1_CATGCTC_L004_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam| awk '{s+=$3+$4} END {print s}'`
   
(first row of the .bam file)  
M_ST-E00201:191:HH7YYALXX:4:1103:31548:29806	16	1	10022	0	79M	*	0	0	CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAAC	]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]Z]]]]]]]]]]	XT:A:R	NM:i:0	X0:i:370	XM:i:0	XO:i:0	XG:i:0	MD:Z:79	XP:i:1  


* get data out of the .bam files:
`samtools mpileup -f hs37d5.fa.gz jar001-b1e1l1p1_CATGCTC_L004_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam | head -5`  
\[mpileup\] 1 samples in 1 input files  
1	10022	C	1	^!,	A  
1	10023	C	1	,	B  
1	10024	C	1	,	E  
1	10025	T	1	,	]  
1	10026	A	1	,	]      
  
col1 - Sequence name (chr)  
col2 - 1-based coordinate  
col3 - Reference base  
col4 - Number of reads covering this position  
col5 - Read bases  
col6 - Base qualities  


### ~~Making it a .vcf file with `mpileup`~~  
`samtools mpileup -f hs37d5.fa.gz -g jar001-b1e1l1p1_CATGCTC_L004_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam|bcftools call -m| sed -n '114,117p'`  
Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid  
\[warning\] samtools mpileup option `g` is functional, but deprecated. Please switch to using bcftools mpileup in future.  
\[mpileup\] 1 samples in 1 input files  

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT  	
1	10022	.	C	.	33.5884	.	DP=1;MQ0F=1;AN=2;DP4=0,1,0,0;MQ=0	GT	0/0  
1	10023	.	C	.	33.5884	.	DP=1;MQ0F=1;AN=2;DP4=0,1,0,0;MQ=0	GT	0/0  
1	10024	.	C	.	33.5884	.	DP=1;MQ0F=1;AN=2;DP4=0,1,0,0;MQ=0	GT	0/0  

### ~~Variant calling with `bcftools` and getting a .vcf~~   
* write the output to 'test.vcf.gz'  
`bcftools mpileup -Ou -f hs37d5.fa.gz jar001-b1e1l1p1_CATGCTC_L004_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam|bcftools call -vmO z -o test.vcf.gz`  
Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid  
\[mpileup\] 1 samples in 1 input files  
* read lines 114-117  
`less test.vcf.gz| sed -n '114,117p'`  
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT 	 
1	649113	.	C	T	8.13869	.	DP=1;SGB=-0.379885;MQ0F=0;AC=2;AN=2;DP4=0,0,0,1;MQ=37	GT:PL	1/1:37,3,0  
1	861630	.	G	A	8.13869	.	DP=1;SGB=-0.379885;MQ0F=0;AC=2;AN=2;DP4=0,0,0,1;MQ=37	GT:PL	1/1:37,3,0  
1	918384	.	G	T	8.13869	.	DP=1;SGB=-0.379885;MQ0F=0;AC=2;AN=2;DP4=0,0,0,1;MQ=37	GT:PL	1/1:37,3,0  


## C) What data I need  
- from the list with genomic locations:  
    - list 1: (Neanderthal & Neanderthal-Denisova) locations
    - list 2: (Human & Human-Denisova) locations
    
*** ADD SCHEMATIC HERE ***

### D) Getting everything together  
#### 1) copy the reference genome from my PC to Uppmax:  
* Local terminal (**don't** log in Uppmax) > `scp address/local_file user@rackham.uppmax.uu.se:/home/usernameRackham/folder`   
#### 2) log into Uppmax & check file location and integrity  
==LOCAL WORK NOT JOB==
#### 3) load necessary modules into Uppmax  
* to see available modules: `module avail`  
* to load modules, such as `samtools`, load `bioinfo-tools` first:  
	`module load bioinfo-tools samtools`  
#### 4) Get the necessary info from .bam files:  
* the .bam files contain strings of DNA with the start location of each string. What I need is the precise location of each nucleotide. That information is obtained by using `samtools mpileup` which extracts this information, besides other details.  
`samtools mpileup -f hs37d5.fa.gz jar001-b1e1l1p1_CATGCTC_L004_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam > jar001.mpileup`  
and  
`samtools mpileup -f hs37d5.fa.gz jar002-b1e1l1p1_ACGCAAC_L005_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam > jar002.mpileup`  

==UPPMAX JOB==  
#### 3) do the script to submit a job, then copy it on Uppmax  
++++++++++++++++++++++++++++++++++  
`#!/bin/bash -l  
#SBATCH -A snic2019-8-150  
#SBATCH -t 22:00:00  
#SBATCH -p core  
  
\# load modules  
module load bioinfo-tools samtools  
  
\# run mpileup to get the necessary positional information  
samtools mpileup -f /home/miba8458/private/workingFolder/raw_internal/hs37d5.fa.gz /home/miba8458/private/workingFolder/raw_internal/jar001-b1e1l1p1_CATGCTC_L004_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam > /home/miba8458/private/workingFolder/raw_internal/jar001.mpileup`  

++++++++++++++++++++++++++++++++++  
* name the file and give it the extension '.sh'  
* (somewhere it was written to make it executable: chmod u+x file.sh, but I did it without it)  
* stay on your local drive and do point 1) again  
* go on Uppmax  
* submit job: `sbatch jar001mpileup.sh `  
* you get an ID for the job, in this case 11608594 and for second file: 11609004
  
#### 4) check the job  
`squeue -u miba8458`  
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)  
          11596631      core jar001mp miba8458 PD       0:00      1 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions)  

`jobinfo -u miba8458`  
Waiting jobs:  
   JOBID    POS PARTITION                      NAME     USER        ACCOUNT ST          START_TIME   TIME_LEFT PRIORITY CPUS NODELIST(REASON)     FEATURES DEPENDENCY  
11596631    823      core          jar001mpileup.sh miba8458 snic2019-8-150 PD                 N/A    22:00:00   100010    1 (Nodes required        (null)   


#### 5) jobs are done  
* check the slurm log files to see how the job went, if there was any problem
`less`
* check the output files  
`less`  
* see their line nr, etc
``  
* copy files from Uppmax to your PC  
`scp user@rackham.uppmax.uu.se:/home/username/some-file local_file_or_directory`  
(To place the file in the directory you are currently standing in, use a dot ('.') as the local directory:  
`scp user@milou.uppmax.uu.se:/home/username/some-file .`)    

#### ~~3) try to use the node of the own HDD~~
https://www.uppmax.uu.se/support/user-guides/how-to-use-the-nodes-own-hard-drive-for-analysis/  
* rewrote the code:  

+++++++++++++++++++++++++++++++  
`#!/bin/bash -l  
#SBATCH -A snic2019-8-150  
#SBATCH -t 22:00:00  
#SBATCH -p core  
  
\# load modules  
module load bioinfo-tools samtools  
  
\# copy the files used in the analysis to \$SNIC\_TMP  
cp /home/miba8458/private/workingFolder/raw_internal/hs37d5.fa.gz /home/miba8458/private/workingFolder/raw_internal/jar001-b1e1l1p1_CATGCTC_L004_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam $SNIC_TMP  
  
cp /home/miba8458/private/workingFolder/raw_internal/hs37d5.fa.gz /home/miba8458/private/workingFolder/raw_internal/jar002-b1e1l1p1_ACGCAAC_L005_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam $SNIC_TMP  
  
\# go to the $SNIC_TMP folder to make sure any temporary files are created there as well  
cd $SNIC_TMP  
  
\# run mpileup to get the necessary positional information  
samtools mpileup -f $SNIC_TMP/hs37d5.fa.gz $SNIC_TMP/jar001-b1e1l1p1_CATGCTC_L004_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam > $SNIC_TMP/jar001TMP.mpileup  
  
samtools mpileup -f $SNIC_TMP/hs37d5.fa.gz $SNIC_TMP/jar001-b1e1l1p1_CATGCTC_L004_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam -> $SNIC_TMP/jar002TMP.mpileup  
  
\# copy the results back to the network file system  
cp $SNIC_TMP/jar001TMP.mpileup /home/miba8458/private/workingFolder/raw_internal/  
cp $SNIC_TMP/jar002TMP.mpileup /home/miba8458/private/workingFolder/raw_internal/  
  
+++++++++++++++++++++++++++++++  
11596876



### G) Check the 3 needed files:  
* the list with genomic locations:  
    * nr of lines:  
    `less -c sima_informative_sites.tab |wc -l`  
    or  
    `less -c sima_informative_sites.tab |awk 'END { print NR }'`
    7 049 547
    * first 5 lines:  
    10	94488	G	A	all  	
    10	95365	T	C	all  	
    10	95830	T	C	all	 
    10	97555	G	A	all	 
    10	97647	G	T	all	 
    * last 5 lines:  
    9	140991238	G	A	all	 
    9	140992405	A	G	all	 
    9	140992739	C	T	Denisova  
    9	140992743	T	C	all	 
    9	140993341	C	T	all	 
* the sequencing lists:  
    * load the samtools module if you are on Uppmax:  
    `module load bioinfo-tools samtools`  
    * nr of lines (without header!):  
    `samtools view jar001-b1e1l1p1_CATGCTC_L004_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam | wc -l`  
    or  
    `samtools view jar001-b1e1l1p1_CATGCTC_L004_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam |awk 'END { print NR }'`  
    469 379 - for 1st    
    and  
    65 691 - for 2nd  
    * first 5 lines for jar001:  
    `samtools view jar001-b1e1l1p1_CATGCTC_L004_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam | head -n 5`  
    M_ST-E00201:191:HH7YYALXX:4:1103:31548:29806	16	1	10022	0	79M	*	0	0	CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAAC	]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]Z]]]]]]]]]]	XT:A:R	NM:i:0	X0:i:370	XM:i:0	XO:i:0	XG:i:0	MD:Z:79	XP:i:1  
    
    M_ST-E00201:191:HH7YYALXX:4:1111:3285:61573	16	1	11980	0	124M	*	0	0	GGGTCTTGATGCTGCGGTCTTCATCTGCAGGTGTCTGACTTCCAGCAACTGCTGGCCTGTGCCAGGGTGCAAGCTGAGCACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAGTGG	]]QVZ]]]]]L]]Z]]]]]]]]]]]]ZZ]]]]]]WLGW]QW=]]]\]]]]]W]QZ]]]QQGG]]V]]ZVW]]+G]L]QWQW]L>LQV:VQ]]ZZ]]]:]]>\]]]\]V]]]]]=Q]:]]VV]QG	XT:A:R	NM:i:1	X0:i:2	X1:i:5	XM:i:1	XO:i:0	XG:i:0	MD:Z:14T109	XP:i:1  
    
    M_ST-E00201:191:HH7YYALXX:4:2220:7872:9044	0	1	13686	0	62M	*	0	0	GGCCATCCGTGAGATCTTCCCAGGGCAGCTCCCCTCTGTGGAATCCAATCTGTCTTCCATCC	]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]	XT:A:R	NM:i:0	X0:i:6	X1:i:1	XM:i:0	XO:i:0	XG:i:0	MD:Z:62	XP:i:1  
    
    M_ST-E00201:191:HH7YYALXX:4:1222:4594:49074	16	1	54317	0	41M	*	0	0	CTGCTTTGTAAATTATAACAGCTCAATTAAGAGAAACCGTA	>]]]]]]]]]]]]]]]]]]]]]]]]]]]Z]]Z]]]]]V]]]	XT:A:R	NM:i:0	X0:i:3	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:41	XA:Z:15,+102476791,41M,0;19,-95927,41M,0;	XP:i:1  
    
    M_ST-E00201:191:HH7YYALXX:4:2107:25986:14547	0	1	66678	20	100M	*	0	0	ACCTGTCAACAGTGGCTGGCTCTTCATGGTTGCTACAATGAGTGTGTAAGATTCTGAAGGACTCCTTTAATAAGCCTAAACTTAATGTTCAACTTAGAAT	]Q]]]]Z]]Z]]]]]Z]]]]]Z]]]]]Z]]]]]]]Z]]]]]]]\]]]]]V]]]]]Z]]V]]V]]Q]]]]]]]]]]]]]]]]]]]]]Z]]]]]]]]]]QV]	XT:A:U	NM:i:0	X0:i:1	X1:i:2	XM:i:0	XO:i:0	XG:i:0	MD:Z:100	XA:Z:15,-102465140,100M,1;19,+108266,100M,1;	XP:i:1  
    
    * last 5 lines for jar001:  
    `samtools view jar001-b1e1l1p1_CATGCTC_L004_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam | tail -n 5`  
    M_ST-E00201:191:HH7YYALXX:4:2113:28239:31142	16	hs37d5	35412866	37	57M	*	0	0	CACTCCCCTCCTCCCAGTTTTACTCCACTCTTTTCCATTCCATTCCATTGCCCTCCA	]]]]V]]V]]]]]]]]]]]]]]Z]Z]]]]]]]]]ZZ]]]]]]]]]]]]]]]]]]]]]	XT:A:U	NM:i:0	X0:i:1	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:57	XP:i:1  
    
    M_ST-E00201:191:HH7YYALXX:4:1215:25489:32707	0	hs37d5	35415637	37	128M	*	0	0	ATTCCATTCAATCCCATTCCACTCTGGTTGATTCCATTCCATTCCATTCCTTCCCATTCCTATCCATTCCTTTCCATTCCATTCCATTCCATTCCATTCCATTCCATTCCCCTCAGGTTGATTCAATT	]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]Z]]]]>]]]]Z]]]]]]]]]]>]]]Z]]]]]]]	XT:A:U	NM:i:0	X0:i:1	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:128	XP:i:1  
    
    M_ST-E00201:191:HH7YYALXX:4:2219:8532:42991	16	hs37d5	35424374	37	90M	*	0	0	GCCAGTGGATCTTAGCTTGCTGTGCTCCATGGGGGTGGGATCTGCTGAGCTAGACGACTTGGCTCCCTGGCTTCAGCCTCCTTTCCAGGG	]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]	XT:A:U	NM:i:0	X0:i:1	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:90	XP:i:1  
    
    M_ST-E00201:191:HH7YYALXX:4:1118:16488:12894	16	hs37d5	35430325	0	45M	*	0	0	CCATTCCATTCCATTCCATTCCATTCCTTTCCATTCCGTTCCATT	]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]	XT:A:R	NM:i:0	X0:i:14	X1:i:699	XM:i:0	XO:i:0	XG:i:0	MD:Z:45XP:i:1  
    
    M_ST-E00201:191:HH7YYALXX:4:2109:23013:20331	16	hs37d5	35445116	37	117M	*	0	0	ATCTTATGTACAGTATAAAATCTATATTTGATGTACTTTCATATTGTGTGTACACAATATAATATATATTGAATGTAAGTTCATATTTTATGTACAGTATATAATGTATAGTTTGGA	]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]Z]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]	XT:A:U	NM:i:2	X0:i:1	X1:i:0	XM:i:2	XO:i:0	XG:i:0	MD:Z:74A41G0	XP:i:1  
    
    * first 5 lines for jar002:  
    `samtools view jar002-b1e1l1p1_ACGCAAC_L005_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam| head -n 5`  
    M_ST-E00201:191:HH7YYALXX:5:2218:27478:34922	16	1	23141	0	52M	*	0	0	CTTCCTAGGTGCCAGGCACTGTTCCATTCCTTTGCATGTTTTGATTAATTTA	]Z]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]	XT:A:R	NM:i:0	X0:i:4	X1:i:1	XM:i:0	XO:i:0	XG:i:0	MD:Z:52XP:i:1  
    
    M_ST-E00201:191:HH7YYALXX:5:1104:30614:57196	0	1	79458	0	35M	*	0	0	GGAAAGAAACAAAGCGCCTGAAGGCTGCACCTGAA	]Q]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]	XT:A:R	NM:i:0	X0:i:4	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:35	XA:Z:6,+69462,35M,0;19,+121055,35M,0;15,-102452843,35M,0;	XP:i:1  
    
    M_ST-E00201:191:HH7YYALXX:5:1215:27752:67357	0	1	110684	0	82M	*	0	0	AGGAACAGAAAACCAAATACAGCATACTCTCAGTTATAAGTGGGAGCTAAATGATGAGAACTCATGAAAACAAAGAATAAAA	]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]Z]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]	XT:A:R	NM:i:1	X0:i:6	X1:i:9	XM:i:1	XO:i:0	XG:i:0	MD:Z:68C13	XP:i:1  
    
    M_ST-E00201:191:HH7YYALXX:5:2206:10115:60817	0	1	137197	0	78M	*	0	0	CTCTGTCCGCGTGGGAGGGGCCGGTGTGAGGCAAGGGCTCACACTGACCTCTCTCAGCGTGGGAGGAGCCAGTGTGAG	]\V]]ZZ]]]]]]]]]]]]]]]]]Z]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]\]]]]]	XT:A:R	NM:i:0	X0:i:5	X1:i:2	XM:i:0	XO:i:0	XG:i:0	MD:Z:78	XP:i:1  
    
    M_ST-E00201:191:HH7YYALXX:5:2215:25367:21034	0	1	245495	0	71M	*	0	0	TCTGCCTCCCAGGTTCAAGTGATTCTCCTGCCTCAGCCGCCCAAGTAGCTGGGATTACAGGTGTCTGCCAC	]\]]]]ZZZ]]]]]]]]]]]]Z]]Z]]]]]Z]]]]]]]]]]]]]]]]]Z]]]]]]]]]]]]]Z]]]]]]]]	XT:A:R	NM:i:0	X0:i:6	X1:i:9	XM:i:0	XO:i:0	XG:i:0	MD:Z:71	XP:i:1  
    
    * last 5 lines for jar002:  
    `samtools view jar002-b1e1l1p1_ACGCAAC_L005_merged.170314_ST-E00201_0191_AHH7YYALXX.hs37d5.fa.cons.90perc.bam| tail -n 5`  
    M_ST-E00201:191:HH7YYALXX:5:2204:23673:20172	0	hs37d5	35333000	37	48M	*	0	0	CTCCATTCCACTCCACTGCACTACTCCCCACTCCACTCCATTCCATTC	]]]]]]]]]]]]]Z]]]]]]]]]]]]]]]]]]]]Z]]]]]]]]]]]]]	XT:A:U	NM:i:0	X0:i:1	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:48	XP:i:1  
    
    M_ST-E00201:191:HH7YYALXX:5:1211:2615:15268	16	hs37d5	35349167	0	45M	*	0	0	GCTTACAGGGGCTATTGTGACATATCTCTGCACTGATCACCCAGG	]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]	XT:A:R	NM:i:1	X0:i:2	X1:i:46	XM:i:1	XO:i:0	XG:i:0	MD:Z:41T3	XP:i:3  
    
    M_ST-E00201:191:HH7YYALXX:5:2119:5477:55350	0	hs37d5	35437046	37	49M	*	0	0	TCATTCAAGTCCTGTCCATTCCATTCCTCTCCATTCCACTCCACTCCAC	]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]	XT:A:U	NM:i:2	X0:i:1	X1:i:0	XM:i:2	XO:i:0	XG:i:0	MD:Z:0C42G5	XP:i:2  
    
    M_ST-E00201:191:HH7YYALXX:5:2216:4929:46947	0	hs37d5	35449519	37	127M	*	0	0	CACTGAGGCTGCAGTCCGGTGGCCCAAACTCAATTAACATTTCATTTCACCCTTCCATTCCATTGCGTTCCATTTCATTCCATTCCATTTCATTCCATTGCATTCCATTCCATTAAATTCCATTGCA	G]\W]W]]]]Z]]]Q]]]]QZ]]]]]]\]W]]]Q]]]]]]]]]]]]VV]]Z]\QWQV]]]]]]V]\V]V]Z]]]]]]]]]]V]]]]Z]]ZVZ]Z]Z]]]]]]]]]QZ]VQ]Z]VQ]]]VVWV\W=Q]	XT:A:U	NM:i:0	X0:i:1	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:127	XP:i:1  
    
    M_ST-E00201:191:HH7YYALXX:5:1107:2879:6829	0	hs37d5	35461699	37	120M	*	0	0	ATAGTGACGGTGATGAGGATTAGGATGATGGTGATGAGGATAGTGATGATGGTCATGGTGATGATGGTGATGAGGATGGTGATGATGGTGATAGTGATGGTGATGATGGTAATGATGATG	Q\V]GZVW]]G]L=W\V]]=W]]]V0V]=]]VGV5]L]LV=VVL]Z>]V5]ZQ+:QQ]V]]LQG]]VL]]]]G]0G=]ZV]Q=]VLQ]]]]]]]Z]]V]]Z]]]]ZZ]VVV]G]Z]]]]]	XT:A:U	NM:i:1	X0:i:1	X1:i:0	XM:i:1	XO:i:0	XG:i:0	MD:Z:7T112	XP:i:1   

* the sequencing lists after `mpileup` extraction:  
    * count the number of lines:  
    `less jar001.mpileup |wc -l`  
    or  
    `less jar001.mpileup |awk 'END { print NR }'`  
    53 588 203 - for 1st  

    `less jar002.mpileup |wc -l`  
    or  
    `less jar002.mpileup |awk 'END { print NR }'`  
    5 692 180 - for 2nd   
    * print first few lines:  
    `less -c jar001.mpileup |head -n 5`  
    1	10022	C	1	^!,	A  
    1	10023	C	1	,	B  
    1	10024	C	1	,	E  
    1	10025	T	1	,	]  
    1	10026	A	1	,	]  

    `less -c jar001.mpileup |tail -n 5`  
    hs37d5	35445228	t	1	,	Z  
    hs37d5	35445229	t	1	,	R  
    hs37d5	35445230	g	1	,	9  
    hs37d5	35445231	g	1	,	1  
    hs37d5	35445232	g	0	*	*  

    `less -c jar002.mpileup |head -n 5`  
    1	23141	C	1	^!,	E  
    1	23142	T	1	,	U  
    1	23143	T	1	,	]  
    1	23144	C	1	,	]  
    1	23145	C	1	,	]  

    `less -c jar002.mpileup |tail -n 5`  
    hs37d5	35461814	t	1	.	]  
    hs37d5	35461815	g	1	.	]  
    hs37d5	35461816	a	1	.	]  
    hs37d5	35461817	t	1	.	T  
    hs37d5	35461818	g	1	.$	E  

* make smaller files to use for making the script (5000 lines):  
    `less jar001.mpileup |sed -n '1,5000p' > jar001.mpileup_5klines`  
    `less jar002.mpileup |sed -n '1,5000p' > jar002.mpileup_5klines`  
    * figure out the last of the loc & position of each file, then crop the list with genomic positions exactly after that loc & position:  
        * last position in jar001.mpileup is:  
        `less jar001.mpileup_5klines | tail -n 1`  
        1	416732	A	1	.	\]  
        * last position in jar002.mpileup is:  
        1	2839409	A	1	.	\]  
        * figure out the position in the genomic document that is one bigger than the biggest one in the sequences (bigger than the last position in jar002, 2 839 409):  
        `less sima_informative_sites.tab |awk '{if ($1=1 && 2839400<$2 && $2 < 2839430) print NR " - " $0;}'`  
        2433871 - 1 2839413 A G all  
        * (for jar001 it is 416732:  
        `less sima_informative_sites.tab |awk '{if ($1=1 && 416725<$2 && $2 < 416760) print NR " - " $0;}'`  
        1951787 - 1 416757 C G all)  
        * copy the positions including 2 839 409 from the genomic positions to a new file:  
        `less sima_informative_sites.tab | sed -n '1,2433871p'>sima_crop.tab`  
            * the table isn't sorted ('-n' numerical sort; '-k' sort by column 2):  
            `less sima_informative_sites.tab |sort -n -k2 > sima_sort3`  
    

### F) Write the python code:  

###  more usefull code:  
* find out if I have a certain duplication in 'pos' column:  
`less sima_informative_sites.tab_sort | awk '{if ($2 == "179281355") print $0;}'`  
  
* print out the lines that are duplicate in columns 1 and 2:  
`less sima_informative_sites.tab_sort | awk 'FNR==NR{a[$1,$2]++;next} a[$1,$2]>1'`  

* crop a large file to the first 5000 lines  
`'less jar001.mpileup |sed -n '1,5000p' > jar001.mpileup_5klines'`  
  
* print the number of the line that matches the query:  
`less duplicated_rows_tab| awk '{if ($2 == "179281355") print NR" : "$0;}'`